Preparing environment with necessary imports

In [1]:
import pandas as pd
%matplotlib inline

Reading in already prepared csv file with data from https://wind-erleben.de <br>
Additionally converting data to correct types and removing columns that are not used in model

In [2]:
df = pd.read_csv('2018.csv', sep='\t')
df.measurement_date = pd.to_datetime(df.measurement_date)
df.drop(columns=["pitch_degrees", "rotor_speed_rpm", "wind_direction_degrees",
                 "rotation_gondola_degrees", "charging_station_w",
                 "battery_drain_or_load_w", "state_of_charge_percent", "battery_voltage_v",
                 "rlm_solar_kw", "slp_solar_kw"], inplace=True)

tidying up data, removing rows that do not really make sense

In [3]:
df = df[abs((df.solar_generation_kw + df.wind_generation_kw + df.chp_kw) - df.total_production_kw) < 1]
df = df[df.power_use_kw != 0]
df = df[(df.total_production_kw + df.electricity_purchase_kw) - df.power_use_kw > -1]
df = df[df.chp_kw < 1000]
df = df[df.electricity_purchase_kw < 10000]

Feature generation

In [4]:
df['green_energy'] = (df.total_production_kw > df.power_use_kw).astype(int)
df['chp_online'] = (df.chp_kw > 10).astype(int)
df['weekday'] = df.measurement_date.dt.weekday
df['time_of_day'] = df.measurement_date.dt.hour
df['month'] = df.measurement_date.dt.month

Creation of data used in model

In [5]:
model_data = df[["time_of_day", "wind_speed_m_s", "chp_online", "green_energy"]]

In [6]:
model_data.describe()

,time_of_day,wind_speed_m_s,chp_online,green_energy
count,4479.00000,4479.000000,4479.000000,4479.000000
mean,11.54722,4.457357,0.326859,0.289797
std,6.87981,2.522608,0.469117,0.453719
min,0.00000,-0.100000,0.000000,0.000000
25%,6.00000,2.500000,0.000000,0.000000
50%,12.00000,4.200000,0.000000,0.000000
75%,17.00000,6.000000,1.000000,1.000000
max,23.00000,19.300000,1.000000,1.000000
